# This pipeline is a fork of DockM8

In [1]:
#Import required libraries and scripts
from DockM8.docking_functions import *
from DockM8.rescoring_functions import *
from DockM8.consensus_methods import *
# from DockM8.scripts.dogsitescorer import *
# from DockM8.scripts.get_pocket import *
from tqdm.autonotebook import tqdm

In [2]:
protein_name  = 'protein_protoss_noligand.pdb'
ligand_library = 'ecft_scores_new_cleaned.sdf'
reference_ligand = 'ref_ligand.pdb'
# snapshot_IDs = ['p9', 'p11']

In [17]:
HERE = Path(_dh[-1])
DATA = (HERE / "data")
# Move input data (protein pdb, docking library and reference ligand) to data directory
software = (HERE / "software")
protein_file = (DATA  / protein_name)
ligand_library = (DATA / ligand_library)
ref_file = (DATA /  reference_ligand)

print(ligand_library)

OUTPUT = DATA / "results"

/home/ibrahim/Github/ECFT-VS-pipeline/data/ecft_scores_new_cleaned.sdf


### Move snapshots to another different folders

### Load Ground truth data with 2D compounds ['ID', '2D structure', 'Activity score']

# Data-preprocessing

### Protein is prepared by [Protoss](https://proteins.plus/)

### Ligand library preparation by Gypsum-DL for 3D conformers generation

In [4]:
from data_preparation import run_gypsumdl


prepared_library_path = OUTPUT / f"{ligand_library.stem}_prepared.sdf"
run_gypsumdl(ligand_library, prepared_library_path)

Molecules are already prepared


In [5]:
df_prepared = PandasTools.LoadSDF(str(prepared_library_path))
df_prepared.head(5)

,ID,ROMol
0,HIPS6128,<rdkit.Chem.rdchem.Mol object at 0x7f0071d7cc80>
1,HIPS449,<rdkit.Chem.rdchem.Mol object at 0x7f00761aab90>
2,HIPS6989,<rdkit.Chem.rdchem.Mol object at 0x7f011035a500>
3,HIPS7002,<rdkit.Chem.rdchem.Mol object at 0x7f0071d7ccf0>
4,HIPS7000,<rdkit.Chem.rdchem.Mol object at 0x7f0071d7cd60>


In [6]:
docking_programs = [
                'GNINA', 
                'SMINA',
                'local_diffdock', 
                'PLANTS', 
                'flexx',
                ]


consensus_methods = []
n_poses = 10
exhaustiveness = 8

# Docking

1. Local DiffDock
2. PLANTS (Implemented by DockM8)
3. SMINA (Implemented by DockM8)
4. GNINA (Implemented by DockM8)
5. FlexX

NOTE : Output of docking step should have at least two columns 
 1. **ID** : Name of the compound , name of docking tool and number of pose e.g. (compoundX_diffdock_01)
 2. **Molecule** : Poses of every docking tool

In [ ]:
from docking import docking
docking(
        docking_programs,
        protein_file,
        prepared_library_path,
        ref_file,
        exhaustiveness,
        n_poses
        )

Extracting ligand coordinates supports either SDF files or PDB files...
Compounds are already docked with GNINA v 1.0
Compounds are already docked with SMINA
Binding pocket is already extracted


Local DiffDock is running ...:   0%|          | 0/212 [00:00<?, ?it/s]

Compound HIPS6128 is already docked with Local DiffDock
Compound HIPS449 is already docked with Local DiffDock
Compound HIPS6989 is already docked with Local DiffDock
Compound HIPS7002 is already docked with Local DiffDock
Compound HIPS7000 is already docked with Local DiffDock
Compound HIPS6994 is already docked with Local DiffDock
Compound HIPS6991 is already docked with Local DiffDock
Compound HIPS6998 is already docked with Local DiffDock
Compound HIPS7006 is already docked with Local DiffDock
Compound HIPS7004 is already docked with Local DiffDock
Compound HIPS7001 is already docked with Local DiffDock
Compound HIPS6992 is already docked with Local DiffDock
Compound HIPS6981 is already docked with Local DiffDock
Compound HIPS6999 is already docked with Local DiffDock
Compound HIPS7242 is already docked with Local DiffDock
Compound HIPS470 is already docked with Local DiffDock
Compound HIPS6984 is already docked with Local DiffDock
Compound HIPS6990 is already docked with Local Dif

# Rescoring

### Choose wanted scoring function from the next list

In [15]:
rescoring = [
    'gnina_rescoring', 
    'ad4',  
    'linf9', 
    'rtmscore', 
    'vinardo', 
    'scorch',
        
    'chemplp', 
    'rfscorevs_v1',
    'rfscorevs_v2',
    'rfscorevs_v3', 
    'vina_hydrophobic', 
    'vina_intra_hydrophobic'
    ]

# Load important dataframes


In [16]:
from rescoring import rescoring_function
  
docked_library_path = OUTPUT / f"allposes.sdf"

rescoring_function(
    rescoring,
    protein_file,
    docked_library_path,
    ref_file,
)

protein is already converted to mol2


Now rescoring with GNINA_RESCORING ... ⌛⌛ 
gnina_rescoring is already excuted
gnina_rescoring is already read


Now rescoring with AD4 ... ⌛⌛ 
ad4 is already excuted
ad4 is already read


Now rescoring with LINF9 ... ⌛⌛ 
linf9 is already excuted
linf9 is already read


Now rescoring with RTMSCORE ... ⌛⌛ 
rtmscore is already excuted
rtmscore is already read


Now rescoring with VINARDO ... ⌛⌛ 
vinardo is already excuted
vinardo is already read


Now rescoring with SCORCH ... ⌛⌛ 
scorch is already excuted
scorch is already read


Now rescoring with CHEMPLP ... ⌛⌛ 
chemplp is already excuted
chemplp is already read


Now rescoring with RFSCOREVS_V1 ... ⌛⌛ 
rfscorevs_v1 is already excuted
rfscorevs_v1 is already read


Now rescoring with RFSCOREVS_V2 ... ⌛⌛ 
rfscorevs_v2 is already excuted
rfscorevs_v2 is already read


Now rescoring with RFSCOREVS_V3 ... ⌛⌛ 
rfscorevs_v3 is already excuted
rfscorevs_v3 is already read


Now rescoring with VINA_HYDROP

# Consensus ranking methods (Implemented by DockM8)
### You can also choose the ranking methods according to you preference

In [7]:
ranking_methods = [
                'best_ECR' ,
                'ECR_average',
                'average_ECR' ,
                'rank_by_rank' ,
                'rank_by_vote' ,
                'best_Zscore',
                'average_Zscore'
                ]

In [8]:
df_rescored = pd.read_csv(str(DATA / 'results' / 'all_rescoring_results.csv')).apply(pd.to_numeric, errors='ignore')
df_scores = PandasTools.LoadSDF(str(ligand_library))[['ID', 'score']]
df_scores = df_scores.rename(columns={'score': 'true_value'})

### Run Ranking methods

In [9]:
from ranking import *


poses_ranking(
    ranking_methods,
    df_rescored,
    OUTPUT,
    df_scores,
)


Number of possible combinations for every ranking method: 1015777
 With total combinations : 7110439
Parallelizing best_ECR...


 92%|█████████▏| 133057/145111 [1:17:28<08:09, 24.61it/s]

Execution time: 4648.840747117996 seconds


 88%|████████▊ | 127380/145111 [1:24:21<09:05, 32.51it/s]

Execution time: 5062.0362293720245 seconds


...

 96%|█████████▌| 139082/145111 [1:28:25<02:41, 37.22it/s]

Execution time: 5306.037352800369 seconds


100%|██████████| 145111/145111 [1:31:00<00:00, 26.57it/s]


Execution time: 5464.490411520004 seconds
Finished average_Zscore...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ibrahim/mambaforge-pypy3/envs/dockm8/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_983724/1857051794.py", line 4, in <module>
    poses_ranking(
  File "/home/ibrahim/Github/ECFT-VS-pipeline/ranking.py", line 150, in poses_ranking
  File "/home/ibrahim/Github/ECFT-VS-pipeline/ranking.py", line 150, in <listcomp>
  File "/home/ibrahim/mambaforge-pypy3/envs/dockm8/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 912, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/ibrahim/mambaforge-pypy3/envs/dockm8/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 577, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/home/ibrahim/mambaforge-pypy3/envs/dockm8/lib/python3.10/site-packages/pandas/io/parsers/readers.py", line 1407, in __init__
    self._engine = self